# 11.1 파이썬 문화에서의 인터페이스와 프로토콜

* 인터페이스 : 시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부
* 프로토콜 : 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스

# 11.2 파이썬은 시퀀스를 찾아낸다

다음 `Foo` 클래스는 `abc.Sequence`를 상속하지 않으며, 시퀀스 프로토콜 메서드 중 `__getitem__()` 메서드만 구현한다.

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0,30,10)[pos]

In [2]:
f = Foo()
f[1]

10

In [3]:
for i in f: print(i)

0
10
20


In [4]:
20 in f

True

`__iter__()` 메서드가 없지만 대체 수단인 `__getitem__()` 메서드가 구현되어 있으므로 반복, `in` 연산자 등이 작동한다.

1장에서 구현한 `FrenchDeck` 클래스도 `abc.Sequence`를 상속하지 않지만, 시퀀스 프로토콜의 `__getitem__()`과 `__len__()` 메서드를 구현한다.

In [5]:
!cat frenchdeck.py

import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, pos):
        return self._cards[pos]


# 11.3 런타임에 프로토콜을 구현하는 멍키 패칭

`FrenchDeck` 클래스에 `shuffle()` 메서드를 구현할 필요가 없는 것은 `random.shuffle()` 함수가 시퀀스 객체 안의 항목들을 섞어주기 때문이다. 그러나 다음과 같이 입력하면 예외가 발생하는데, 이는 `FrenchDeck` 객체가 할당을 지원하지 않기 때문이다.

In [9]:
from random import shuffle
from frenchdeck import FrenchDeck
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

파이썬은 동적 언어이므로 코드를 대화형 콘솔에서 실행하는 동안에도 이 문제를 수정할 수 있다.

In [11]:
def set_card(deck, pos, card):
    deck._cards[pos] = card
    
FrenchDeck.__setitem__ = set_card

In [12]:
shuffle(deck)

In [13]:
deck[:5]

[Card(rank='4', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='2', suit='hearts'),
 Card(rank='A', suit='diamonds'),
 Card(rank='7', suit='clubs')]

이런 방법을 멍키 패칭(소스 코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위)라고 한다.

# 11.4 알렉스 마르텔리의 물새

ABC...?

특정한 클래스의 속성을 갖춘 클래스를 만들고 싶을 땐 ABC를 상속하거나 등록하라. 그리고 어떤 속성을 가지고 있는지 보여주고 싶을 떈 `isinstance(self, cls)` (단 cls는 ABC) 를 써라

# 11.5 ABC 상속하기

`FrenchDeck2`를 `colletions.MutableSequence`의 서브클래스로 선언한다.

In [35]:
import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    # MutableSequence를 상속하므로 이 클래스의 추상 메서드인 
    # __delitem__() 도 구현해야함
    def __delitem__(self, position):
        del self._cards[position]
        
    # insert() 또한 추상 메서드
#     def insert(self, position, value):
#         self._cards.insert(position, value)

파이썬은 모듈을 로딩하거나 컴파일할 때가 아니라 실제로 객체를 생성할 때 추상 메서드의 구현 여부를 확인한다.

In [36]:
frenchdeck2 = FrenchDeck2()

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract methods __getitem__, __setitem__, insert

따라서 쓰지 않더라도 `__delitem__()`과 `insert()` 메서드를 구현해야 한다.

# 11.6 표준 라이브러리의 ABC

대부분의 ABC는 `collections.abc` 모듈에 정의되어 있다. 어떤 ABC들이 있는지 살펴보자.

## 11.6.1 collections.abc의 ABC

collections.abc에 들어 있는 ABC에 대한 UML 다이어그램

![collections.abc](https://user-images.githubusercontent.com/33891164/39363928-e9ba4520-49e0-11e8-9cc5-273134e16df4.png)

## 11.6.2 ABC의 숫자탑

`numbers` 패키지는 다음과 같이 계층 구조로 이루어져 있다. `Number`가 최상위 슈퍼클래스이며, `Integral`까지 내려간다.

* Number
* Complex
* Real
* Rational
* Integral

정수형인지 검사해야 하는 경우 `isinstance(x, numbers.Integral)`을 이용하면 된다.

In [43]:
import numbers

a = 1+ 2j
b = 3+ 3j
a*b, isinstance(a, numbers.Complex)

((-3+9j), True)

# 11.7 ABC의 정의와 사용

ABC 클래스를 생성해보자. 여기서는 유한 집합에서 무작위로 항목을 선택하도록 설계된 기계를 만들 것인데, 선택하는 방법을 두 가지로 구현할 것이기 때문에 ABC 클래스 생성 후 상속하는 방식으로 만들어보자.

`Tombola` ABC는 두 개의 추상 메서드와 두 개의 구상 메서드를 가진다.

* `load()` : 항목을 컨테이너 안에 넣는다.
* `pick()` : 컨테이너 안에서 무작위로 항목 하나를 꺼내서 반환한다.

* `loaded()` : 컨테이너 안에 항목이 하나 이상 들어 있으면 `True` 반환한다.
* `inspect()` : 내용물을 변경하지 ㅇ낳고 현재 컨테이너 안에 들어있는 항목을 튜플로 만들어 반환한다.

In [18]:
# tombola.py

import abc

class Tombola(abc.ABC): # ABC 정의하기 위해 상속
    
    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
        
    @abc.abstractmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 'LookupError'가 발생한다.
        """
        
    def loaded(self): # ABC 에도 구상 메서드가 들어갈 수 있다
        """최소 한 개의 항목이 있으면 True, 아님 False 반환"""
        # ABC의 구상 메서드는 반드시 ABC에 정의된 인터페이스, 즉
        # ABC의 다른 구상 메서드나 추상 메서드, 혹은 프로퍼티만 사용해야 한다.
        return bool(self.inspect())
    
    def inspect(self):
        """현재 안에 있는 항목들로 구성된 정렬된 튜플 반환"""
        items = []
        while True:
            try:
                # pick() 을 계속 호출해서 Tombola 객체를 비움
                items.append(self.pick())
            except LookupError:
                break
        self.load(items) # load 메서드를 호출해서 다시 넣는다
        return tuple(sorted(items))

`inspect()` 메서드는 멍청해보이지만, 이 코드의 핵심은 ABC 안에서 인터페이스에 정의된 다른 메서드만 이용하는 한 ABC에 구상 메서드를 제공하는 것도 가능하다는 점을 보여주는 것이다. 물론 상속하면서 오버라이드 해도 된다.

`loaded()` 연산자는 상당히 값비싼 연산을 수행하므로 구상 서브클래스에서 더 잘 구현하는 것이 좋다.

`LookupError`를 선택한 이유는 `Tombola`의 서브클래스의 데이터 구조체에서 발생할 가능성이 높은 `IndexError`와 `KeyError` 에러의 상위 클래스이기 때문이다.

* `IndexError` : 시퀀스에서 마지막 이후 인덱스를 호출 시 발생
* `KeyError` : 매핑에 존재하지 않는 키로 호출 시 발생

실제로 ABC가 인터페이스 검사를 제대로 수행하는지 확인해보자.

In [19]:
from tombola import Tombola
class Fake(Tombola):
    def pick(self):
        return 13

In [20]:
Fake

__main__.Fake

In [22]:
f = Fake()
# load()를 구현하지 않았으므로 객체 생성 시 TypeError 발생 ()

TypeError: Can't instantiate abstract class Fake with abstract methods load

## 11.7.1 ABC 상세 구문

ABC 선언 시 `abc.ABC` 나 다른 ABC를 상속하는 방법이 가장 좋다. 그러나 이는 파이썬 3.4에 추가되었으므로, 파이썬3의 이전 버전을 사용하고 있으면 다음과 같이 사용해야 한다.

```python
class Tombola(metaclass=abc.ABCMeta):
    # ...
```

파이썬2 에서는 `__metaclass__` 클래스 속성을 사용해야 한다.

```python
class Tombola(object): # 파이썬2 코드
    __metaclass__ = abc.ABCMeta
    # ...
```

`@abstractmethod` 외에도 `abc` 모듈은 `@abstractclassmethod`, `@abstractstaticmethod`, `@abstractproperty` 데커레이터를 정의한다. 그러나 얘네는 파이썬 3.3 이후 사용 중단 안내되었으므로, 다음과 같이 데커레이터를 쌓아 올려서 사용하자.

```python
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass
```

근데... 잘되는데? ㅋㅋㅋㅋㅋㅋ 뭐지

In [23]:
# tombola.py

import abc

class Tombola(abc.ABC): # ABC 정의하기 위해 상속
    
    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
        
    @abc.abstractclassmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 'LookupError'가 발생한다.
        """
        
    def loaded(self): # ABC 에도 구상 메서드가 들어갈 수 있다
        """최소 한 개의 항목이 있으면 True, 아님 False 반환"""
        # ABC의 구상 메서드는 반드시 ABC에 정의된 인터페이스, 즉
        # ABC의 다른 구상 메서드나 추상 메서드, 혹은 프로퍼티만 사용해야 한다.
        return bool(self.inspect())
    
    def inspect(self):
        """현재 안에 있는 항목들로 구성된 정렬된 튜플 반환"""
        items = []
        while True:
            try:
                # pick() 을 계속 호출해서 Tombola 객체를 비움
                items.append(self.pick())
            except LookupError:
                break
        self.load(items) # load 메서드를 호출해서 다시 넣는다
        return tuple(sorted(items))

In [27]:
class Fake(Tombola):
    def pick(self):
        return 13
    def load(self):
        pass

In [28]:
Fake

__main__.Fake

In [29]:
f = Fake()

## 11.7.2 Tombola ABC 상속하기

Tombola 인터페이스를 만족시키는 구상 서브클래스를 만들어보자. 여기서 `random.SystemRandom()`은 다음 속성을 가진다.

    운영 체제에서 제공하는 소스에서 난수를 생성하기 위해 os.urandom() 함수를 사용하는 클래스. 모든 시스템에서 사용 가능한 것은 아닙니다. 소프트웨어 상태에 의존하지 않으며, 시퀀스는 재현되지 않습니다. 따라서, seed() 메서드는 효과가 없으며, 무시됩니다. getstate()와 setstate() 메서드는 호출되면 NotImplementedError를 발생시킵니다.

In [31]:
# bingo.py

import random
from tombola import Tombola

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [32]:
# lotto.py

import random
from tombola import Tombola

class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable) # 사본으로 저장
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            pos = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingCage')
        return self._balls.pop(pos)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

`BingoCage`는 섞고 마지막을 꺼내는 반면 `LotteryBlower`는 임의의 원소를 꺼낸다. 

이후는 가상 서브클래스 내용인데...
Too Much... 같다...